In [1]:
import pandas as pd
import cx_Oracle
import pickle

In [2]:
ip = '172.16.80.74'
port = '1521'
SID = 'RPTDB'

dsn_tns = cx_Oracle.makedsn(ip, port, SID)
connection = cx_Oracle.connect('ANLYUSR', 'ANLYUSR$', dsn_tns)

In [3]:
CRIF = ['TBL_TRN_MB_HIGH_ACCOUNTSUMMARY','TBL_TRN_MB_HIGH_INDVRESPONSE','TBL_TRN_MB_HIGH_INDVRESP_ADDR','TBL_TRN_MB_HIGH_INDVRESP_ID','TBL_TRN_MB_HIGH_REQUEST','TBL_TRN_MB_HIGH_SCORE',]

In [5]:
for i in CRIF:
    query = "SELECT TB3.PROPNO,sum(status) as {} \
    FROM \
    (SELECT TB1.PROPNO,decode(TB2.PROSPECT_NO,NULL,0,1) as status \
    FROM \
    (SELECT PROPNO \
    FROM TBL_TRN_CUSTOMER_DETAILS CUS \
    WHERE CUS.AGMT_DATE > TO_DATE('31-OCT-18', 'DD-MON-YY') \
    AND CUS.PORTFOLIO='CD') TB1 LEFT OUTER JOIN \
    (SELECT PROSPECT_NO FROM BIUSR.{} \
    WHERE CREATED_DATE > TO_DATE('31-OCT-18', 'DD-MON-YY') \
    AND SUBSTR(PROSPECT_NO,5,2) = 'CD') TB2 \
    ON TB1.PROPNO = TB2.PROSPECT_NO) TB3 \
    GROUP BY TB3.PROPNO".format(i,i)

    Data = pd.read_sql(query, con=connection, chunksize=10000)
    
    data = pd.concat([chunk for chunk in Data])
    
    if i == 'TBL_TRN_MB_HIGH_ACCOUNTSUMMARY':
        DF_TBL_TRN_MB_HIGH_ACCOUNTSUMMARY = data
    elif i == 'TBL_TRN_MB_HIGH_INDVRESPONSE':
        DF_TBL_TRN_MB_HIGH_INDVRESPONSE = data
    elif i == 'TBL_TRN_MB_HIGH_INDVRESP_ADDR':
        DF_TBL_TRN_MB_HIGH_INDVRESP_ADDR = data
    elif i == 'TBL_TRN_MB_HIGH_INDVRESP_ID':
        DF_TBL_TRN_MB_HIGH_INDVRESP_ID = data
    #elif i == 'TBL_TRN_MB_HIGH_INDVRESP_PHONE':
    #    DF_TBL_TRN_MB_HIGH_INDVRESP_PHONE = data
    elif i == 'TBL_TRN_MB_HIGH_REQUEST':
        DF_TBL_TRN_MB_HIGH_REQUEST = data
    elif i == 'TBL_TRN_MB_HIGH_SCORE':
        DF_TBL_TRN_MB_HIGH_SCORE = data
    print("---------------------------"+i+"----DONE-------------------")
    

---------------------------TBL_TRN_MB_HIGH_ACCOUNTSUMMARY----DONE-------------------
---------------------------TBL_TRN_MB_HIGH_INDVRESPONSE----DONE-------------------
---------------------------TBL_TRN_MB_HIGH_INDVRESP_ADDR----DONE-------------------
---------------------------TBL_TRN_MB_HIGH_INDVRESP_ID----DONE-------------------
---------------------------TBL_TRN_MB_HIGH_REQUEST----DONE-------------------
---------------------------TBL_TRN_MB_HIGH_SCORE----DONE-------------------


In [6]:
DF_Data = pd.merge(DF_TBL_TRN_MB_HIGH_ACCOUNTSUMMARY, DF_TBL_TRN_MB_HIGH_INDVRESPONSE, on='PROPNO', how='inner')

In [7]:
DF_Data = pd.merge(DF_Data, DF_TBL_TRN_MB_HIGH_INDVRESP_ADDR, on='PROPNO', how='inner')

In [8]:
DF_Data = pd.merge(DF_Data, DF_TBL_TRN_MB_HIGH_INDVRESP_ID, on='PROPNO', how='inner')

In [9]:
DF_Data = pd.merge(DF_Data, DF_TBL_TRN_MB_HIGH_REQUEST, on='PROPNO', how='inner')

In [10]:
DF_Data = pd.merge(DF_Data, DF_TBL_TRN_MB_HIGH_SCORE, on='PROPNO', how='inner')

In [11]:
DF_Data.head()

,PROPNO,TBL_TRN_MB_HIGH_ACCOUNTSUMMARY,TBL_TRN_MB_HIGH_INDVRESPONSE,TBL_TRN_MB_HIGH_INDVRESP_ADDR,TBL_TRN_MB_HIGH_INDVRESP_ID,TBL_TRN_MB_HIGH_REQUEST,TBL_TRN_MB_HIGH_SCORE
0,3000CD0043323,1,1,0,0,1,1
1,3000CD0048516,1,1,0,0,1,1
2,3000CD0049443,1,1,0,0,1,1
3,3000CD0049455,1,1,2,2,1,1
4,3000CD0049677,1,1,2,1,1,1


In [12]:
DF_Data.to_csv("D:\CD\CD_MB_CRIF_6_months.csv")